# Hente hemmeligheter fra secret manager

In [ ]:
from google.cloud import secretmanager
from google.oauth2 import service_account

In [ ]:
credentials = service_account.Credentials.from_service_account_file('/home/jupyter/sa_key.json')

In [ ]:
secrets = secretmanager.SecretManagerServiceClient(credentials=credentials)

In [ ]:
project_name = "knada-gcp"
secret_name = "vebjorn-rekkebo-bac1"
version = "latest"

resource_name = f"projects/{project_name}/secrets/{secret_name}/versions/{version}"
secret = secrets.access_secret_version(name=resource_name)

In [ ]:
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

In [ ]:
import psycopg2

import pandas as pd
import datetime

from sqlalchemy import create_engine

In [ ]:
method = "postgresql+psycopg2"
host = "A01DBVL029.adeo.no:5432"
db_name = "pensjon-psak-db"

alchemyEngine = create_engine(f'{method}://{secrets["PSAK_POSTGRES_USER"]}:{secrets["PSAK_POSTGRES_PASSWORD"]}@{host}/{db_name}')
dbConnection = alchemyEngine.connect()

In [ ]:
df_pref = pd.read_sql(
    """
    select bruk_nytt_design, count(*) Antall
    from t_saksbeh_preferanser
    group by bruk_nytt_design;
    """,
    dbConnection).T

In [ ]:
new_header = df_pref.iloc[0]
df_pref = df_pref[1:]
df_pref.columns = new_header
df_pref["Uttrekk_tidspunkt"] = datetime.datetime.utcnow()

df_pref.columns = ["Uspesifisert", "Gammelt", "Nytt", "Uttrekk_tidspunkt"]
df_pref = df_pref.reset_index(drop=True)

df_pref["Uspesifisert"] = pd.to_numeric(df_pref["Uspesifisert"])
df_pref["Gammelt"] = pd.to_numeric(df_pref["Gammelt"])
df_pref["Nytt"] = pd.to_numeric(df_pref["Nytt"])

In [ ]:
df_pref["Andel_nytt"] = df_pref.Nytt / df_pref[["Uspesifisert", "Gammelt", "Nytt"]].sum(axis=1)
df_pref["Andel_nytt"] = df_pref["Andel_nytt"].apply(lambda x: round(x, 2))

In [ ]:
df_pref.head()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig, SchemaField, enums

table_id = f'pensjon-saksbehandli-prod-1f83.saksbehandling_psak.nytt_gammelt_design'
job_config = LoadJobConfig(
    schema = [
        SchemaField("Uspesifisert", enums.SqlTypeNames.INTEGER),
        SchemaField("Gammelt", enums.SqlTypeNames.INTEGER),
        SchemaField("Nytt", enums.SqlTypeNames.INTEGER),
        SchemaField("Uttrekk_tidspunkt", enums.SqlTypeNames.TIMESTAMP),
        SchemaField("Andel_nytt", enums.SqlTypeNames.FLOAT),
    ],
    write_disposition="WRITE_APPEND",
)

client = Client(project="pensjon-saksbehandli-prod-1f83", credentials=credentials)

job = client.load_table_from_dataframe(df_pref, table_id, job_config=job_config)
job.result()